In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from bokeh.plotting import figure, output_notebook,show

# define model
def distill(x,t,rr,feed,x_feed):
    # Input (3):
    # 1. Reflux ratio is the Manipulate variable
    #    Reflux ration (L/D)
    #    rr = p(1)
    
    # Disturbance variables (DV)
    # Feed flowrate (mol/min)
    # feed = p(2)
    
    # Mole fraction of feed
    # x_feed = p(3)
    
    # States (32):
    # x(0)-Reflux Drum Liquid Mole Fraction of Component A
    # x(1)-Tray 1 - Liquid Mole Fraction of Component A
    #.
    #.
    #.
    # x(16) - Tray 16 - Liquid Mole Fraction of Component A
    #.
    #.
    #.
    # x(30) - Tray 30 - Liquid Mole Fraction of Component A
    # x(31) - Reboiler Liquid Mole Fraction of Component A
    
    #----------------------------------
    # parameters
    # Distillate flowrate (mol/min)
    D = 0.5*feed
    # Flowrate of the liquid in the rectification section
    L = rr*D
    # Vapor flowrate in the column (mol/min)
    V = L + D
    # Flowrate of the liquid in the stripping section (mol/min)
    FL = feed + L
    # Relative volatility = (yA/xA)/(yB/xB) = KA/KB = alpha(A,B)
    vol = 1.6
    # Total molar holdup on each tray
    atray = 0.25
    # Total molar holdup in the condenser
    acond = 0.5
    # Total molar holdup in the reboiler
    areb = 1.0
    # Vapor mole fractions of component A
    # From the equilibrium assumption and mole balances
    #1) vol = (yA/xA)/(yB/xB)
    #2) xA+xB = 1
    #3) yA+yB = 1
    y = np.empty(len(x))
    for i in range(32):
        y[i] = x[i]*vol/(1.0+(vol-1.0)*x[i])
    
    # Compute xdot
    xdot = np.empty(len(x))
    xdot[0] = 1.0/acond*V*(y[1]-x[0])
    for i in range(1,16):
        xdot[i] = 1.0/atray*(L*(x[i-1]-x[i])-V*(y[i]-y[i+1]))
    xdot[16] = 1.0/atray*(feed*x_feed+L*x[15]-FL*x[16]-V*(y[16]-y[17]))
    for i in range(17,31):
        xdot[i] = 1.0/atray*(FL*(x[i-1]-x[i])-V*(y[i]-y[i+1]))
    xdot[31] = 1.0/areb*(FL*x[30]-(feed-D)*x[31]-V*y[31])
    return xdot

# Steady state initial conditions for the 32 states
x_ss =np.array([0.935,0.900,0.862,0.821,0.779,0.738,
                0.698,0.661,0.628,0.599,0.574,0.553,0.535,0.521,
                0.510,0.501,0.494,0.485,0.474,0.459,0.441,0.419,
                0.392,0.360,0.324,0.284,0.243,0.201,0.161,0.125,
                0.092,0.064])


x0 = x_ss

# steady state initial condition
rr_ss = 3.0

# time interval (min)
t = np.linspace(0,10,100)

# store results for plotting
xd = np.ones(len(t))*x_ss[0]
rr = np.ones(len(t))*rr_ss
ff = np.ones(len(t))
xf = np.ones(len(t))*0.5
sp = np.ones(len(t))*0.97

# step in reflux ratio
rr[10:] = 3.5
rr[50:] = 6.0
rr[80:] = 9.0

# feed concentration (mol/rac)
xf[50:] = 0.42

# feed flow rate
ff[80:] = 1.0

# simulate
for i in range(len(t)-1):
    ts = [t[i],t[i+1]]
    y = odeint(distill,x0,ts,args=(rr[i],ff[i],xf[i]))
    xd[i+1] = y[-1][0]
    x0 = y[-1]
    
# Construct results and save data file
# Column 1 = time
# Column 2 = reflux ratio
# Column 3 = distillate composition
data = np.vstack((t,rr,xd)) # vertical stack
data = data.T             # transpose data
np.savetxt('data.txt',data,delimiter=',')

# Plot the results
output_notebook()
p1 = figure(plot_width = 400,plot_height = 300,y_axis_label=r'$RR$',x_axis_label = 'Time (min)')
p1.line(t,rr,line_color = 'blue',line_dash = '4 4',line_width = 3,legend = 'Reflux ratio')

p2 = figure(plot_width = 400,plot_height = 300,y_axis_label='Feed',x_axis_label = 'Time (min)')
p2.line(t,xf,line_color = 'black',line_dash = '2 2',line_width = 3,legend = 'Feed composition')
p2.line(t,ff,line_color = 'green',line_width = 3,legend = 'Feed flow (mol/min)')

p3 = figure(plot_width = 400,plot_height = 300,y_axis_label=r'$x_d\;(mol/L)$',x_axis_label = 'Time (min)')
p3.line(t,xd,line_color = 'red',line_width = 3,legend = 'Distillate composition')
p3.line(t,sp,line_color = 'black',line_dash = '1 1',line_width = 3,legend = 'Set point')

show(p1)
show(p2)
show(p3)


    

Loading BokehJS ...

In [12]:
# Estimated model with FOPDT model
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from bokeh.plotting import figure, output_notebook,show

# import csv data file
# column 1 = time (t)
# column 2 = input (u)
# column 3 = output (yp)
data = np.loadtxt('data.txt',delimiter=',')
t = data[:,0]
u = data[:,1]
yp = data[:,2]
# initial conditions
u0 = u[0]
yp0 = yp[0]

# interpolate time with pump
uf = interp1d(t,u)
# fopdt model
def model(y,t,uf,Km,taum,thetam):
    # arguments
    # y = output
    # t = time
    # uf = input linear function(for time shift)
    # Km = model gain
    # taum = model time constant
    # thetam = model time constant
    # time-shift u
    try:
        if (t-thetam)<0:
            um=uf(0.0)
        else:
            um=uf(t-thetam)
    except:
        #print('Error with time extrapolation: '+str(t))
        um=0
    dydt = (-y + Km*um)/taum
    return dydt
    
# simulate FOPDT model with x = [Km,taum,thetam]
def sim_model(x):
    # input_arguments
    Km = x[0]
    taum = x[1]
    thetam = x[2]
    # storage for model values
    ym = np.zeros(len(t)) 
    # initial condition
    ym[0] = yp0
    # loop through time steps
    for i in range(0,len(t)-1):
        ts = [t[i],t[i+1]]
        y1 = odeint(model,ym[i],ts,args=(uf,Km,taum,thetam))
        ym[i+1]=y1[-1]
    return ym

def objective(x):
    ym = sim_model(x)
    obj = 0.0
    for i in range(len(ym)):
        obj = obj + (ym[i]-yp[i])**2
    return obj
x0 = [1.0,1.0,1.0]
print('Initial SSE Objective: ' + str(objective(x0)))

bnds = ((0.0,1.0e10),(0.0,1.0e10),(0.0,1.0e10))
solution = minimize(objective,x0,method = 'SLSQP',bounds = bnds)
print(solution)
x = solution.x

ym1 = sim_model(x0)
ym2 = sim_model(x)

output_notebook()
p1 = figure(plot_width = 600,plot_height = 400,y_axis_label='Distillate composition',x_axis_label = 'Time (min)')
# p1.line(t,ym1,line_color = 'blue',line_width = 3,legend = 'Initial guess')
p1.line(t,ym2,line_color = 'red',line_dash = '4 4',line_width = 3,legend = 'Optimized')
p1.line(t,yp,line_color = 'green',line_width = 3,legend = 'Process data')

show(p1)



Initial SSE Objective: 395.29663788
     fun: 0.0001627257439769504
     jac: array([  9.45984182e-04,  -2.33427618e-05,   3.72430241e-04,
         0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 32
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([  0.30333575,  31.92099143,   0.4983749 ])


Loading BokehJS ...

In [29]:
# Implement PID controller
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from bokeh.plotting import figure, output_notebook,show

# define model
def distill(x,t,rr,feed,x_feed):
    # Input (3):
    # 1. Reflux ratio is the Manipulate variable
    #    Reflux ration (L/D)
    #    rr = p(1)
    
    # Disturbance variables (DV)
    # Feed flowrate (mol/min)
    # feed = p(2)
    
    # Mole fraction of feed
    # x_feed = p(3)
    
    # States (32):
    # x(0)-Reflux Drum Liquid Mole Fraction of Component A
    # x(1)-Tray 1 - Liquid Mole Fraction of Component A
    #.
    #.
    #.
    # x(16) - Tray 16 - Liquid Mole Fraction of Component A
    #.
    #.
    #.
    # x(30) - Tray 30 - Liquid Mole Fraction of Component A
    # x(31) - Reboiler Liquid Mole Fraction of Component A
    
    #----------------------------------
    # parameters
    # Distillate flowrate (mol/min)
    D = 0.5*feed
    # Flowrate of the liquid in the rectification section
    L = rr*D
    # Vapor flowrate in the column (mol/min)
    V = L + D
    # Flowrate of the liquid in the stripping section (mol/min)
    FL = feed + L
    # Relative volatility = (yA/xA)/(yB/xB) = KA/KB = alpha(A,B)
    vol = 1.6
    # Total molar holdup on each tray
    atray = 0.25
    # Total molar holdup in the condenser
    acond = 0.5
    # Total molar holdup in the reboiler
    areb = 1.0
    # Vapor mole fractions of component A
    # From the equilibrium assumption and mole balances
    #1) vol = (yA/xA)/(yB/xB)
    #2) xA+xB = 1
    #3) yA+yB = 1
    y = np.empty(len(x))
    for i in range(32):
        y[i] = x[i]*vol/(1.0+(vol-1.0)*x[i])
    
    # Compute xdot
    xdot = np.empty(len(x))
    xdot[0] = 1.0/acond*V*(y[1]-x[0])
    for i in range(1,16):
        xdot[i] = 1.0/atray*(L*(x[i-1]-x[i])-V*(y[i]-y[i+1]))
    xdot[16] = 1.0/atray*(feed*x_feed+L*x[15]-FL*x[16]-V*(y[16]-y[17]))
    for i in range(17,31):
        xdot[i] = 1.0/atray*(FL*(x[i-1]-x[i])-V*(y[i]-y[i+1]))
    xdot[31] = 1.0/areb*(FL*x[30]-(feed-D)*x[31]-V*y[31])
    return xdot

# Steady state initial conditions for the 32 states
x_ss =np.array([0.935,0.900,0.862,0.821,0.779,0.738,
                0.698,0.661,0.628,0.599,0.574,0.553,0.535,0.521,
                0.510,0.501,0.494,0.485,0.474,0.459,0.441,0.419,
                0.392,0.360,0.324,0.284,0.243,0.201,0.161,0.125,
                0.092,0.064])


x0 = x_ss

Kp = 0.3033
taup = 31.92
thetap = 0.4983

Kc = 20.0
tauI = 4.0
tauD = 4.0

# time interval (min)
t = np.linspace(0,100,1000)

# store results for plotting
# process variable
xd = np.ones(len(t))*x_ss[0]
xd0 = xd[0]
# manipulated variabel reflux ratio
rr = np.ones(len(t))*rr_ss
# disturbance variables
ff = np.ones(len(t))
ff[200:] = 1.4
ff[400:] = 0.6
ff[600:] = 1.0
xf = np.ones(len(t))*0.5
xf[300:] = 0.55
xf[600:] = 0.45
xf[800:] = 0.5

# set point sp=0.97
sp = 0.97

# controller bias
ubias = rr_ss

# upper and lower limit
u_hi = 10.0
u_lo = 1.0

# PID results
P = np.zeros(len(t))
I = np.zeros(len(t))
D = np.zeros(len(t))
ie = np.zeros(len(t))
id = np.zeros(len(t))

# simulate
for i in range(len(t)-1):
    delta_t = t[1]-t[0]
    error = sp - xd0
    P[i] = Kc*error
    if i>=1:
        ie[i] = ie[i-1] + error * delta_t
        id[i] = (xd[i]-xd[i-1])/delta_t
    I[i] = Kc/tauI * ie[i]
    D[i] = Kc*tauD * id[i]
    u = ubias + P[i] + I[i] + D[i]
    if u>=u_hi:
        u = u_hi
        ie[i] = ie[i] + error * delta_t
    if u<=u_lo:
        u = u_lo
        ie[i] = ie[i] + error * delta_t
    rr[i+1] = u
    ts = [t[i],t[i+1]]
    y = odeint(distill,x0,ts,args=(u,ff[i],xf[i]))
    xd[i+1] = y[-1][0]
    xd0 = xd[i+1]
#     print('--------------')
#     print(xd0)
    x0 = y[-1]
#     print(x0)

# Plot the results
output_notebook()
p1 = figure(plot_width = 400,plot_height = 300,y_axis_label=r'$RR$',x_axis_label = 'Time (min)')
p1.line(t,rr,line_color = 'blue',line_dash = '4 4',line_width = 3,legend = 'Reflux ratio')

p2 = figure(plot_width = 400,plot_height = 300,y_axis_label='Feed',x_axis_label = 'Time (min)')
p2.line(t,xf,line_color = 'black',line_dash = '2 2',line_width = 3,legend = 'Feed composition')
p2.line(t,ff,line_color = 'green',line_width = 3,legend = 'Feed flow (mol/min)')

p3 = figure(plot_width = 400,plot_height = 300,y_axis_label=r'$x_d\;(mol/L)$',x_axis_label = 'Time (min)')
p3.line(t,xd,line_color = 'red',line_width = 3,legend = 'Distillate composition')
p3.line(t,sp,line_color = 'black',line_dash = '1 1',line_width = 3,legend = 'Set point')

show(p1)
show(p2)
show(p3)

Loading BokehJS ...